In [1]:
from ipyleaflet import Map, basemaps, Circle, Polyline, Marker, AwesomeIcon # Stuff for the map
import gpxpy
import gpxpy.gpx

class map_handler:
    def __init__(self, zoom):
        self.m = Map(basemap=basemaps.OpenStreetMap.Mapnik, zoom=2)
        self.m.layout.height = '700px'
        self.gateway_reception_layers = [] # contains reception "circles"
        self.track_layer = None # contains a polyline that represents the loaded gpx track
        self.location_marker_layer = None
        self.center = None
        self.track = []
        self.gateway_pos = set()
        display(self.m)
        
    def add_gateway_receptions(self, r): # r is a list of ((lat, lon), radius)
        for (loc, radius) in r:
            if loc not in self.gateway_pos:
                self.gateway_pos.add(loc)
                
                ic = AwesomeIcon(name='dot-circle-o', marker_color='green', icon_color='black', spin=False)
                self.m.add_layer(Marker(icon=ic, location = loc))

            c = Circle()
            c.location = loc
            c.radius = radius
            c.color = 'red'
            c.fill_color = 'red'
            self.gateway_reception_layers.append(c)
            self.m.add_layer(c)
            
        self.center_map(loc)
            
    def clear_gateway_receptions(self):
        for c in self.gateway_reception_layers:
            self.m.remove_layer(c)
        self.gateway_reception_layers = []
        
    def center_map(self, loc):
        if self.center == None: #map is centered only once
            self.m.center = loc
            self.m.zoom = 13
            self.center = loc
            
    def load_gpx_track(self, gpxdata): # gpxdata is a string containing the xml data
        gpx = gpxpy.parse(gpxdata)
        
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    self.track.append([point.latitude, point.longitude, point.time])
        line = Polyline(locations=[[a[0], a[1]] for a in self.track], color='#3366FF', fill=False)
        
        self.track_layer = line
        self.m.add_layer(line)
        
    def clear_gpx_track(self):
        self.m.remove_layer(self.track_layer)
        
    # This adds a marker to the map based on the GPX track with a timestamp closest to t
    def show_current_pos(self, t):
        if self.location_marker_layer != None:
            self.m.remove_layer(self.location_marker_layer)
            self.location_marker_layer = None
            
        if not self.track:
            return
        
        # get index of closest track point
        (idx, track_entry) = min(enumerate(map_hnd.track), key=lambda x: abs(x[1][2].timestamp()-t))
        timediff = abs(map_hnd.track[idx][2].timestamp()-t)

        # if the difference is larger than 5s, we do not associate t with track point and return
        if timediff > 5:
            return
        
        self.location_marker_layer = Marker(location = (track_entry[0], track_entry[1]))
        self.m.add_layer(self.location_marker_layer)
        